![Screenshot 2023-02-22 164309.png](Screenshot_2023-02-22_164309.png)

I 1978 lurte 3 smarte menn fra MIT på hvordan en fiktiv bank kunne bruke en server (time-shared service) til å kalkulere sensitive data fra sine låntakere uten å gi fra seg noe informasjon. Et av forslagene var å bruke homomorfisk kryptering.

Dette var kun teoretisk i 30 år før Gentry laget det først FHE rammeverket i 2009. I 2011 kom Microsoft ut med et paper “Can Homomorphic Encryption be Practical?”. I 2018 kom Microsoft ut med en implementasjon kalt Simple Encrypted Arithmetic Library (SEAL). Og i 2022 kom Edge med Password Monitor som bruker FHE til å sjekke om passordet ditt finnes i lekkede passord-databaser. (Chrome kom kort tid etterpå med en hash-basert løsning).


## RSA
1. Bob selects two large prime $p<<0$ and $q<<0$ and calclate $n=p \cdot q$
2. Bob calculate $\phi(n) = (p-1)(p+2)$. Choose $e$ such that $\gcd(e, \phi(n))=1$ (Or just a prime number works also fine).
3. Bob calculate private key $d$ to be the modular inverse of $e$, in other words $de \equiv 1 (\mod \phi(n))$.
4. Bob sends the public key $p_k = n,p$
5. Anna encrypts message $m$ to ciphertext $c \equiv m^e (\mod n)$. And sends it.
6. Bob computes the message $c^d \equiv m (\mod n)$

In [7]:
# Bobs computer
p, q, e = 11, 17, 3 # pretend that p and 1 are large (4096 bits)
n = p * q
print(f'{n=}')
phi_n = (p-1)*(q-1)
d = pow(e, -1, mod = phi_n)

# Annas computer
m_a = 42
c = m_a**e % n
print(f'{c=}')

# Bobs computer
m_b = c**d % n
print(f'{m_b=}')


n=187
c=36
m_b=42
